# Notebook 1: Systematically Improving Your RAG Application

> **Note** : This notebook is a preview of what we cover in [improvingrag.com](https://improvingrag.com). Stop settling for "Looks Good to Me" and start building better systems. Learn how to turn RAG from a risky experiment into a structured, data-driven practice. Check out [improvingrag.com](https://improvingrag.com) for a proven foundational framework to help you go beyond the basics to improve performance, quality, and user experience. 

In this notebook, you'll learn why focusing solely on the generation step can be costly and slow down improvements, and how optimizing the retrieval process can lead to faster, more effective iterations.

## Why this matters

When developers build RAG applications, they often focus on the generation step. This is an expensive choice and prevents them from iterating fast and improving the quality of their application. Without a clear systematic approach in mind, it's hard to know how generic advice to use a specific re-ranker or embedding model actually translates to improvements in retrieval quality.

## What you'll learn

In this notebook, we'll cover the following topics:

1. Understanding RAG's Importance
- Explore how large language models struggle without the proper context.
- Learn why adding relevant context through methods like vector search is critical for effective retrieval.

2. Where Semantic Search Falls Short
- Identify common failure modes when relying only on semantic search.
- Understand how metadata filters can enhance search results and why cosine similarity may miss key nuances in queries.

3. Embracing a Systematic Approach
- Discover the value of quantifying the impact of each component in your RAG pipeline.
- See why starting with retrieval evaluation can lead to faster, more cost-effective improvements and higher user satisfaction.

By the end of this notebook, you'll have a clear understanding of why a systematic approach to building RAG applications is key to building any high quality RAG application and how starting with retrieval evaluation leads to faster, cheaper iteration.

## What is RAG

Retrieval Augmented Generation (RAG) is a technique whereby we inject some information into the prompt of a language model so that it can use it to answer a question about information that it wasn't trained on. This is crucial for helping to adapt language models to specific domains and use cases.

Let's see an example below.

### Why Context Matters

Language models often have a fixed knowledge cut off date and do not have access to any private information on your specific domain. Imagine we're building a RAG application to answer questions about a clothing store - without any context, the model will not know the price or other details of any specific item in your catalog.

To illustrate this, let's see an example below where we ask a language model to answer a question about the price of a fictional item in our catalog called AX123 which has a price of $49.99 and is in stock at the moment.

In [1]:
import openai
import instructor
from pydantic import BaseModel
from rich import print


class Response(BaseModel):
    response: str


client = instructor.from_openai(openai.OpenAI())

print(
    client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "What's the price of AX123? Is it in stock?"},
        ],
        response_model=Response,
    )
)

Response(
    response="I'm sorry, but I don't have real-time access to information regarding product prices or stock 
availability. You may want to check an online retailer or contact the store directly for the most accurate 
information."
)

Obviously this doesn't work because our model doesn't know anything about this specific item we're asking it about. In order to fix this, we can inject some additional context into the model. 

In [2]:
print(
    client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """
                You are a helpful assistant that can answer questions about a clothing store.
                
                Here is the price list and availability of the items in the store:
                AX123: $49.99, In Stock
                BX456: $29.99, Out of Stock
                CX789: $79.99, In Stock
                DX012: $39.99, In Stock
                """,
            },
            {"role": "user", "content": "What's the price of AX123? Is it in stock?"},
        ],
        response_model=Response,
    )
)

Response(response='The price of AX123 is $49.99 and it is In Stock.')

We can see that with this additional context, our model is able to answer the question correctly. 

This makes RAG a powerful technique for businesses that need to:

1. Keep product information current without constantly retraining models
2. Handle dynamic data like pricing, inventory, and product specifications
3. Provide accurate customer service across thousands or millions of items

With a clear understanding of RAG and the importance of contextual data, the next step is to see how we can retrieve that context efficiently. In the next section, we’ll explore vector search—an essential tool for breaking down documents and finding the most relevant information quickly.

### Vector Search

Most people try to make sure that they can provide the relevant context for their model by using vector search. This is done by breaking down our documents into relevant chunks and then embedding each chunk. When the user makes a query, we then embed the query and use a similarity search to find the most similar items in our vector database.

Let's see an example where we use lancedb to embed the query and then use a similarity search to find the most similar items in the database. 

We'll do so by declaring a LanceModel which will be used to define the schema of the table in lancedb. We'll then ingest in some sentences into the table and then use a similarity search to find the most similar items in the database.

In [3]:
from lancedb import connect
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

db = connect("./lancedb")
table_name = "show"
func = get_registry().get("openai").create(name="text-embedding-3-small")


class Item(LanceModel):
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()


table = db.create_table(table_name, schema=Item, mode="overwrite")

table.add(
    [
        {
            "text": "The Witcher follows Geralt of Rivia, a monster hunter with supernatural abilities"
        },
        {
            "text": "Yennefer of Vengerberg is a powerful sorceress and one of the main characters in The Witcher"
        },
        {
            "text": "The continent is plagued by monsters and political intrigue in The Witcher series"
        },
        {
            "text": "Ciri is a princess with elder blood who becomes Geralt's ward in The Witcher"
        },
    ]
)

/Users/ivanleo/Documents/coding/ai-engineer-summit/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


When we make a query, we'll then fetch the top 3 items that are most similar to the query.

In [4]:
table.search(query="Who is the main character in the Witcher?").select(["text"]).limit(
    2
).to_list()

[{'text': 'The Witcher follows Geralt of Rivia, a monster hunter with supernatural abilities',
  '_distance': 0.6419593691825867},
 {'text': 'Yennefer of Vengerberg is a powerful sorceress and one of the main characters in The Witcher',
  '_distance': 0.6808716058731079}]

Using these retrieved items, we can then use a language model to generate a response. But this doesn't always work, and often times, semantic search fails to retrieve the right items. For instance, if we add a short sentence to the query, we can see that semantic search no longer retrieves the chunk of `The Witcher follows Geralt of Rivia, a monster hunter with supernatural abilities` within the top-2 items itself.



In [5]:
table.search(
    query="There are many important figures and princesses in the Witcher but who is the main character in the Witcher?"
).select(["text"]).limit(2).to_list()

[{'text': 'Yennefer of Vengerberg is a powerful sorceress and one of the main characters in The Witcher',
  '_distance': 0.6996006369590759},
 {'text': "Ciri is a princess with elder blood who becomes Geralt's ward in The Witcher",
  '_distance': 0.7565385699272156}]

We've seen how vector search falls short when we have slightly more complex queries. Now let's examine some high level scenarios where semantic search struggles and why relying solely on cosine similarity may lead to unexpected results below.

## When Semantic Search Falls Short

Because Semantic Search relies on the cosine similarity between the query and the documents, it sometimes fails to understand specific nuances of the queries. As a result, there are a few categories of queries that we expect semantic search to fail for.

Let's see some examples of these queries that we expect semantic search to fail for. We'll do so using a dataset that I've cleaned ahead of time at `ivanleomk/ai-engineer-summit-ecommerce-taxonomy`. This dataset contains a list of products from a clothing store along with some associated metadata.

We'll start by loading in the dataset first into our LanceDB database before showing some of the key failure modes of semantic search.

In [6]:
from datasets import load_dataset

dataset = load_dataset("ivanleomk/ai-engineer-summit-ecommerce-taxonomy")["train"]

In [7]:
from lancedb.pydantic import LanceModel, Vector
import lancedb
from lancedb.embeddings import get_registry

# Create Embedding Function
func = get_registry().get("openai").create(name="text-embedding-3-small")


# Define a Model that will be used as the schema for our collection
class Item(LanceModel):
    id: int
    title: str
    description: str = func.SourceField()
    brand: str
    category: str
    product_type: str
    attributes: str
    material: str
    pattern: str
    price: float
    vector: Vector(func.ndims()) = func.VectorField()
    in_stock: bool


db = lancedb.connect("./lancedb")
table_name = "items"


table = db.create_table(table_name, schema=Item, mode="overwrite")
entries = []
for row in dataset:
    entries.append(
        {
            "id": row["id"],
            "title": row["title"],
            "description": row["description"],
            "brand": row["brand"],
            "category": row["category"],
            "product_type": row["product_type"],
            "attributes": row["attributes"],
            "material": row["material"],
            "pattern": row["pattern"],
            "price": row["price"],
            "in_stock": row["in_stock"],
        }
    )

table.add(entries)

table = db.open_table(table_name)
print(f"{table.count_rows()} rows in the table")

191 rows in the table


Because semantic search relies on the cosine similarity between the query and the documents, it sometimes fails to understand specific nuances of the queries. As a result, there are a few categories of queries that we expect semantic search to fail for.

1. Queries for complement items - Eg. `I'd like a shirt that goes well with my blue polyster skirt`
2. Queries for items that have specific attributes - Eg. `I'm looking for a blue t-shirt that's under $50`
3. Queries for items that have specific constraints - Eg. `I'm looking for a skirts that's in size S that I can wear to a party tonight`

Let's see how semantic search performs for these queries and how we end up retrieving the wrong documents. We'll write a small helper function here that will take in a query and then retrieve the top 10 items from the table as a pandas dataframe.

In [8]:
def retrieve_items(query,table):
    df = table.search(query=query).select(["title","description","category","material","product_type","price","in_stock"]).limit(10).to_pandas()
    df = df.drop(columns=['_distance'])
    return df

retrieve_items("Any nice jeans?",table)

,title,description,category,material,product_type,price,in_stock
0,High-Waist Blue Jeans,These high-waist blue jeans are a staple for a...,Bottoms,Denim,Jeans,231.27,False
1,High-Waisted Blue Jeans,These classic high-waisted blue jeans offer a ...,Bottoms,Denim,Jeans,362.77,False
2,High-Waisted Relaxed Jeans,These high-waisted jeans offer a relaxed fit a...,Bottoms,Denim,Jeans,288.76,True
3,High Rise Skinny Jeans,Elevate your denim collection with these class...,Bottoms,Denim,Jeans,382.00,False
4,High-Rise Skinny Jeans,These elegant pink high-rise skinny jeans offe...,Bottoms,Denim,Jeans,144.55,True
5,Olive Green Skinny Jeans,"Crafted from stretchy cotton, these olive gree...",Bottoms,Cotton,Jeans,392.11,False
6,Mid Rise Skinny Jeans,Experience the perfect fit with these mid-rise...,Bottoms,Denim,Jeans,11.87,True
7,High-Waisted Skinny Jeans,These high-waisted skinny jeans are a wardrobe...,Bottoms,Denim,Jeans,31.08,True
8,Mid Rise Skinny Jeans,These mid-rise skinny jeans offer a flattering...,Bottoms,Denim,Jeans,103.01,False
9,Women's High Waisted Denim Shorts,"The ultimate in casual chic, these high-waiste...",Bottoms,Denim,Shorts,389.12,True


### Complement Items

Let's try a query of `I have a blue t-shirt at the moment and I'd love a pair of jeans to go with it`.

We can see that for Complement items, even though we're asking for a pair of jeans, we end up retrieving mostly T-Shirts.

In [9]:
retrieve_items("I have a blue t-shirt at the moment and I'd love a pair of jeans to go with it",table)

,title,description,category,material,product_type,price,in_stock
0,High-Waist Blue Jeans,These high-waist blue jeans are a staple for a...,Bottoms,Denim,Jeans,231.27,False
1,High-Waisted Blue Jeans,These classic high-waisted blue jeans offer a ...,Bottoms,Denim,Jeans,362.77,False
2,Thunderstorm Print Jeans,Make a bold statement with these black jeans b...,Bottoms,Denim,Jeans,397.68,False
3,Denim High-Waisted Shorts,"Versatile and comfortable, these denim high-wa...",Bottoms,Denim,Shorts,390.63,False
4,Floral V-Neck T-Shirt,This stylish navy V-neck t-shirt features a bo...,Tops,Cotton,T-Shirts,319.97,False
5,Graphic Logo T-Shirt,This Levi's graphic logo t-shirt features a bo...,Tops,Cotton,T-Shirts,354.38,False
6,Striped Crew Neck T-Shirt,This classic striped tee features a timeless c...,Tops,Cotton,T-Shirts,139.20,True
7,Women's Classic Grey V-Neck T-Shirt,This versatile grey V-neck T-shirt offers both...,Tops,Cotton,T-Shirts,344.87,True
8,Printed Yellow T-Shirt,Brighten your wardrobe with this vibrant yello...,Tops,Cotton,T-Shirts,90.49,True
9,Basic Crew Neck T-Shirt,This classic yellow crew neck t-shirt offers a...,Tops,Cotton,T-Shirts,204.79,False


Most of the items that were retrieved are skirts, which is not what we're looking for. Remember the the query is asking for a shirt, but because the query starts with the word `skirt`, semantic search will retrieve items that are related to skirts.

### Specific Attributes

Now let's see how well semantic search performs for queries that have specific attributes.

In [10]:
retrieve_items("I want a skirt that's under $150 which is made of Cotton",table)

,title,description,category,material,product_type,price,in_stock
0,Women's White Sleeveless Top with Skirt,This elegant white sleeveless top pairs perfec...,Tops,Cotton,Tank Tops,262.91,False
1,White Eyelet Mini Skirt,"Featuring a delicate eyelet design, this white...",Bottoms,Cotton,Skirts,102.31,True
2,Floral Print Skirt,Flaunt your femininity with this charming flor...,Bottoms,Cotton,Skirts,300.24,False
3,Rust Midi Skirt,This chic rust-colored midi skirt offers a sop...,Bottoms,Cotton,Skirts,338.49,False
4,High-Waisted Pencil Skirt,This elegant high-waisted pencil skirt is desi...,Bottoms,Polyester,Skirts,64.18,True
5,Sleeveless Button-Down Blouse,This classic sleeveless button-down blouse is ...,Tops,Cotton,Blouses,353.57,False
6,Women's High-Waisted Midi Skirt,Make a statement with this chic high-waisted m...,Bottoms,Polyester,Skirts,389.11,False
7,Striped Midi Skirt,Add a touch of elegance to your ensemble with ...,Bottoms,Polyester,Skirts,349.57,True
8,Women's Classic Grey V-Neck T-Shirt,This versatile grey V-neck T-shirt offers both...,Tops,Cotton,T-Shirts,344.87,True
9,Black Denim Skirt,A versatile black denim skirt with front pocke...,Bottoms,Denim,Skirts,289.67,True


We can see here that the model is able to retrieve mostly skirts, but there are 4 of them which are not made of cotton. Additionally, for our query, although we specified that we want an item that's under $150, we end up retrieving only 2 items that are under $150, of which only one item is made of cotton.

### Avaliability Constraints

Now let's see how well semantic search does when we specify constraints on the availability of the item.

In [11]:
retrieve_items("I want a skirt that's in stock now",table)

,title,description,category,material,product_type,price,in_stock
0,Green Plaid Mini Skirt,Add a pop of pattern to your outfit with this ...,Bottoms,Polyester,Skirts,191.17,True
1,Women's Pleated Midi Skirt,Add a pop of color to your outfit with this vi...,Bottoms,Polyester,Skirts,318.64,False
2,Floral Print Skirt,Flaunt your femininity with this charming flor...,Bottoms,Cotton,Skirts,300.24,False
3,Black Denim Skirt,A versatile black denim skirt with front pocke...,Bottoms,Denim,Skirts,289.67,True
4,Rust Midi Skirt,This chic rust-colored midi skirt offers a sop...,Bottoms,Cotton,Skirts,338.49,False
5,Women's High-Waisted Midi Skirt,Make a statement with this chic high-waisted m...,Bottoms,Polyester,Skirts,389.11,False
6,Plaid Pencil Skirt,This plaid pencil skirt is a versatile additio...,Bottoms,Cotton,Skirts,275.42,False
7,High-Waisted Pencil Skirt,This elegant high-waisted pencil skirt is desi...,Bottoms,Polyester,Skirts,64.18,True
8,White Eyelet Mini Skirt,"Featuring a delicate eyelet design, this white...",Bottoms,Cotton,Skirts,102.31,True
9,Women's White Sleeveless Top with Skirt,This elegant white sleeveless top pairs perfec...,Tops,Cotton,Tank Tops,262.91,False


Similar to the previous examples, we can see that the model is able to retrieve the right category of items here but we don't have any way to filter out items that are not in stock. 

In fact, out of the 10 retrieved items, 50% of them are not in stock. 

Having identified these limitations, the natural question arises: How can we overcome these pitfalls? The answer lies in integrating metadata filters. In the following section, we’ll demonstrate how applying these filters can significantly enhance the relevance of our retrieval results

### Improving Search Accuracy with Metadata Filters

One of the ways that we can get around this issue is to implement metadata filters on the retrieved items after we've retrieved them. Let's see how we can do this.

We can do this with lanceDB that supports filtering on the items out of the box. Let's revisit our previous example and see how we can implement some filters on the retrieved items.

Let's revisit our previous example and see how we can implement these filters.

#### Complement Items

Let's revisit our previous example where we wanted a shirt that goes well with a blue polyester skirt. We can ensure we have the right items by applying a filter on the category of the item itself.

In [12]:
import pandas as pd

items = [
    {
        "title": item["title"],
        "description": item["description"],
        "category": item["category"],
        "material": item["material"],
        "product_type": item["product_type"],
        "price": item["price"],
    }
    for item in table.search(
        query="I have a blue t-shirt at the moment and I'd love a pair of jeans to go with it"
    )
    .limit(10)
    .where("product_type='Jeans'", prefilter=True)
    .to_list()
]

pd.DataFrame(items)

,title,description,category,material,product_type,price
0,High-Waist Blue Jeans,These high-waist blue jeans are a staple for a...,Bottoms,Denim,Jeans,231.27
1,High-Waisted Blue Jeans,These classic high-waisted blue jeans offer a ...,Bottoms,Denim,Jeans,362.77
2,Thunderstorm Print Jeans,Make a bold statement with these black jeans b...,Bottoms,Denim,Jeans,397.68
3,Olive Green Skinny Jeans,"Crafted from stretchy cotton, these olive gree...",Bottoms,Cotton,Jeans,392.11
4,High-Waisted Skinny Jeans,These high-waisted skinny jeans are a wardrobe...,Bottoms,Denim,Jeans,31.08
5,High Rise Skinny Jeans,Elevate your denim collection with these class...,Bottoms,Denim,Jeans,382.00
6,High-Waisted Relaxed Jeans,These high-waisted jeans offer a relaxed fit a...,Bottoms,Denim,Jeans,288.76
7,High-Rise Skinny Jeans,These elegant pink high-rise skinny jeans offe...,Bottoms,Denim,Jeans,144.55
8,Mid Rise Skinny Jeans,These mid-rise skinny jeans offer a flattering...,Bottoms,Denim,Jeans,103.01
9,Mid Rise Skinny Jeans,Experience the perfect fit with these mid-rise...,Bottoms,Denim,Jeans,11.87


By applying the right filter on our retrieved items, we can ensure that we're only retrieving the right items. This is important because it ensures that we're providing the model with the relevant context that it needs to answer the question correctly.

#### Specific Attributes

Now let's revisit the previous example where we wanted a skirt that's under $150 which is made of cotton. We can ensure we have the right items by applying a filter on the material of the item itself.

In [13]:
import pandas as pd

items = [
    {
        "title": item["title"],
        "description": item["description"],
        "category": item["category"],
        "material": item["material"],
        "product_type": item["product_type"],
        "price": item["price"],
    }
    for item in table.search(
        query="I want a skirt that's under $150 which is made of Cotton"
    )
    .limit(10)
    .where(
        "material='Cotton' AND price < 150 and product_type='Skirts'", prefilter=True
    )
    .to_list()
]

pd.DataFrame(items)

,title,description,category,material,product_type,price
0,White Eyelet Mini Skirt,"Featuring a delicate eyelet design, this white...",Bottoms,Cotton,Skirts,102.31


#### Availability Constraints

Now let's see the last example where we wanted a skirt that's in stock now. We can ensure we have the right items by applying a filter on the in_stock attribute of the item itself.

In [14]:
import pandas as pd

items = [
    {
        "title": item["title"],
        "description": item["description"],
        "category": item["category"],
        "material": item["material"],
        "product_type": item["product_type"],
        "price": item["price"],
    }
    for item in table.search(query="I want a skirt that's in stock now")
    .limit(10)
    .where("in_stock=True AND product_type='Skirts'", prefilter=True)
    .to_list()
]

pd.DataFrame(items)

,title,description,category,material,product_type,price
0,Green Plaid Mini Skirt,Add a pop of pattern to your outfit with this ...,Bottoms,Polyester,Skirts,191.17
1,Black Denim Skirt,A versatile black denim skirt with front pocke...,Bottoms,Denim,Skirts,289.67
2,High-Waisted Pencil Skirt,This elegant high-waisted pencil skirt is desi...,Bottoms,Polyester,Skirts,64.18
3,White Eyelet Mini Skirt,"Featuring a delicate eyelet design, this white...",Bottoms,Cotton,Skirts,102.31
4,Striped Midi Skirt,Add a touch of elegance to your ensemble with ...,Bottoms,Polyester,Skirts,349.57


We can see that by applying the right filter on our retrieved items, we can ensure that we're only retrieving the right items. This is important because it ensures that we're providing the model with the relevant context that it needs to answer the question correctly.

By having metadata filters on hand, we can ensure that we're providing the model with the right context that it needs to answer the question correctly. 

This is very common in many RAG applications where we might have chunks that have 

1. Different Permissions - only specific users have access to certain documents
2. Different Document Types - if a user is asking for information about signed proposals, then we only want to retrieve chunks from proposals that have been signed for instance
3. Different Time Periods - if a user is asking for information about a specific time period, then we only want to retrieve chunks from that time period. For instance if we have a stock application, and the user asks about stock prices between February 1st and February 10th, then we only want to retrieve chunks from that time period. We can't be generating a response based on stock prices from last year.

In all of these cases, we can ensure that we're providing the model with the right context that it needs to answer the question correctly by applying the right filters on the retrieved items. 




## Why A Systematic Approach Matters

We need a structured approach to building our RAG applications because it provides us with

1. A clear way to evaluate different technologies and techniques
2. A decision making process that allows us to prioritise different components of our pipeline
3. A methodology to diagnose and improve application performance
4. A set of standardized metrics and benchmarks to measure success

In short, with a systematic approach, we can waste less time guessing about whether something works or not and instead use objective metrics to guide our decisions. This ensures that we can quantify and know the impact of each component in our pipeline.

The real goal here is that instead of 

1. Being told to make the AI better
2. Not knowing what to do when they say look at the data
3. Having no idea what features to prioritise 

We can instead

1. Identify key metrics that we can track to determine what's working and what's not
2. Segment customer queries to understand what development efforts to prioritise
3. Generate and bootstrap evaluation datasets to test the impact of different components of our pipeline so we know what exactly works for us.

In short, your only job here is to make sure that you're applying consistent effort and the same system to achieve the boring success that you're looking for.

### Starting with Retrieval

If you're an engineer working on a RAG application, you're probably focusing on generation. We've already seen how just relying on semantic search alone fails to retrieve the right items in certain cases, so why not focus on retrieval first?

When we think about it carefully: 

1. Without the right context, the generation step is essentially useless
2. We're missing out on major big wins that we could get just from improving retrieval alone

Here's a quick breakdown of the tradeoffs between evaluating generation and retrieval

| Aspect | Content Generation | Retrieval Metrics |
|--------|-------------------|-------------------|
| Speed | 1-10s per test | 10-800ms per test |
| Cost | $100s per run | Negligible |
| Objectivity | Subjective | Quantitative |
| Iteration Speed | Hours | Minutes |
| Scale | Limited | Automated |

This is because Retrieval is not only multi-step, but also multi-index​. 

![](./assets/rag.png)

Without clearly understanding the impact of each component in our retrieval pipeline, we're essentially flying blind and hoping for the best. 

With models having stronger reasoning ability and longer context windows, it's important to make sure that we nail our retrieval by making sure that we can consistently retrieve the right context for our model. This is an incredibly important step that is often overlooked.

## Conclusion

In this notebook, we've explored why building effective RAG applications requires a systematic approach rather than just implementing semantic search. Through hands-on examples with our e-commerce dataset, we saw how even simple queries - like finding matching items, filtering by price, or checking stock levels - quickly expose the limitations of pure semantic search.

These challenges highlight two key insights:
1. Simply retrieving semantically similar text often misses important details and constraints
2. Metadata filtering and structured data are crucial for handling real-world queries effectively

We'll put some of these principles in practice in the next notebook where we'll explore how to evaluate our retrieval pipeline. We'll walk through some key metrics that we can use to evaluate our retrieval pipeline, show how we can use synthetic queries to compute an initial baseline before benchmarking how techniques like better item descriptions impact performance.

If you enjoyed this notebook, please consider checking out the [course](https://improvingrag.com) which walks through these principles in greater detail.